In [1]:
from src.data import extract_frames_realtime
import matplotlib.pyplot as plt
import cv2
from ultralyticsplus import YOLO, render_result
from datetime import datetime

In [2]:
model = YOLO('ultralyticsplus/yolov8n')

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

100%|██████████| 6.23M/6.23M [01:01<00:00, 106kB/s] 


In [3]:
def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

In [4]:
video_path = "data/traffic.mp4"

capture = cv2.VideoCapture(video_path)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
capture_fps = round(capture.get(cv2.CAP_PROP_FPS))

current_frame = 0

writer = create_video_writer(capture, "output.mp4")

while capture.isOpened():
    start = datetime.now()
    
    ret, frame = capture.read()

    if not ret:
        break

    detections = model(frame)[0]
    
    for data in detections.boxes.data.tolist():
        confidence = data[4]
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])
        cv2.rectangle(frame, (xmin, ymin) , (xmax, ymax), (255, 50, 30), 2)

    end = datetime.now()
    elapsed = (end - start).total_seconds()
    
    print(f"Time elapsed for processing 1 frame: {elapsed * 1000:.0f}ms")

    fps = f"FPS: {1 / elapsed:.2f}"
    cv2.putText(frame, fps, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (30, 50, 255), 8)
    
    # cv2_imshow(frame)
    writer.write(frame)
    
    current_frame += 1

    if current_frame == 300:
        break



capture.release()
writer.release()

Ultralytics YOLOv8.0.43  Python-3.11.4 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs



NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

CPU: registered at C:\Users\circleci\project\torchvision\csrc\ops\cpu\nms_kernel.cpp:112 [kernel]
QuantizedCPU: registered at 

In [ ]:
# index = 0
# for frame in frame_gen:
#     plt.imshow(frame, interpolation="bilinear")
#     plt.axis("off")
#     plt.show()
#     index += 1

#     if index == 5:
#         break
